#Preparation

Mount your Google Drive when using Google Colab. This is not needed when running this notebook locally.

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


Download sparsenet package from GitHub. Note you need to have Python 3 (and NumPy) and TensorFlow 2.0 or above installed first. Commands under Linux and Colab:




In [6]:
%%bash
# pip install --upgrade tensorflow
cd /content/drive/My\ Drive/packages
git clone https://github.com/datapplab/sparsenet.git

Cloning into 'sparsenet'...


Alternatively, you can pip install sparsenet package from GitHub. This will also install the dependencies, eg TensorFlow and NumPy. 

In [ ]:
!pip install git+https://github.com/datapplab/sparsenet

Import dependencies for this notebook. Please edit the path to sparsenet package downloaded. You don't need the `sys.path.append` line if the package was installed instead.

In [7]:
import pandas as pd
import numpy as np
import gc
import random
import os

# import tf 2.x
import tensorflow as tf
print(tf.__version__)

import sys
sys.path.append('/content/drive/My Drive/packages/sparsenet')
from sparsenet.core import sparse

2.3.0


#Real single cell RNA-Seq dataset--BSEQ

[BSEQ](https://shenorrlab.github.io/bseqsc/vignettes/bseq-sc.html) is a scRNA-Seq dataset on individual human pancreatic islets cells. This processed dataset include 1822 cells and the top 1000 genes with the biggest variance.

## Data preparation

Load and prepare the BSEQ dataset, and split it into training and validation sets by ratio of 2/3 and 1/3. Note testing set is the same as validation set here. Again, please edit the path to sparse module installed.

In [9]:
# change the path to your own
infile= "/content/drive/My Drive/packages/sparse/data/bseq.tsv"
bseq1 = pd.read_csv(infile,sep="\t", header=0, index_col=0)
X = bseq1.values
nfs=X.shape[1]
y=X[:,nfs-1] -1
X = X[:,:nfs-1]
y=y.astype(int)
ns=y.size
idx=np.arange(ns)
np.random.seed(1)
idx1=np.random.choice(idx, size=ns, replace=False)
X=X[idx1]
y=y[idx1]
n_train = round(ns*2/3)
n_val = round(ns*1/3)
X_train = X[:n_train]
X_val   = X[n_train:n_train+n_val]
X_test= X_val
y_train = y[:n_train]
y_val   = y[n_train:n_train+n_val]
y_test=y_val
n_values = np.max(y_train) + 1


## Classic dense neural network or multilayer perceptron (MLP)

Build a `tf.keras.Sequential` model with two dense layers. Alternatively, you can use 1 or 2 sparse layers in place of the dense layer(s) as in commented code. Choose an optimizer and loss function for training.

In [10]:
tf.keras.backend.clear_session()
gc.collect()

nunits=250
dens=0.1
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=X_train.shape[1]),
  tf.keras.layers.Dense(nunits, activation=None),
  tf.keras.layers.Dropout(0.2),
  # sparse(units=nunits, density=dens, activation=None),
  tf.keras.layers.Dense(10, activation='softmax')
  # sparse(units=10, density=0.4, activation='softmax'),
])

lr=1e-3
optimizer = tf.keras.optimizers.Nadam(lr =lr)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=64, 
          validation_data=(X_test, y_test))


Epoch 1/10
19/19 [==============================] - 0s 11ms/step - loss: 23.6434 - accuracy: 0.3210 - val_loss: 14.1590 - val_accuracy: 0.3773
Epoch 2/10
19/19 [==============================] - 0s 4ms/step - loss: 11.4835 - accuracy: 0.5210 - val_loss: 8.6665 - val_accuracy: 0.5783
Epoch 3/10
19/19 [==============================] - 0s 5ms/step - loss: 3.6074 - accuracy: 0.7572 - val_loss: 1.8743 - val_accuracy: 0.8929
Epoch 4/10
19/19 [==============================] - 0s 5ms/step - loss: 2.9038 - accuracy: 0.8206 - val_loss: 0.7987 - val_accuracy: 0.9012
Epoch 5/10
19/19 [==============================] - 0s 4ms/step - loss: 0.4306 - accuracy: 0.9226 - val_loss: 0.1792 - val_accuracy: 0.9736
Epoch 6/10
19/19 [==============================] - 0s 4ms/step - loss: 0.3315 - accuracy: 0.9358 - val_loss: 0.1511 - val_accuracy: 0.9720
Epoch 7/10
19/19 [==============================] - 0s 4ms/step - loss: 0.1733 - accuracy: 0.9663 - val_loss: 0.1617 - val_accuracy: 0.9753
Epoch 8/10
19/19

Model summary info, note the number of parameters is much bigger than that in model with sparse layers below.

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 250)               250250    
_________________________________________________________________
dropout (Dropout)            (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2510      
Total params: 252,760
Trainable params: 252,760
Non-trainable params: 0
_________________________________________________________________


## Sparse neural network or multilayer perceptron (MLP)

Build a `tf.keras.Sequential` model with two sparse layers. Alternatively, you can use 1 dense layer as in the commented code or 2 dense layers as shown above. 

In [11]:
tf.keras.backend.clear_session()
gc.collect()

nunits=250
dens1=0.1
dens2=0.4
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=X_train.shape[1]),
  sparse(units=nunits, density=dens1, activation=None),
  # tf.keras.layers.Dense(10, activation='softmax')
  sparse(units=10, density=dens2, activation='softmax'),
])

lr=1e-3
optimizer = tf.keras.optimizers.Nadam(lr =lr)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=64, 
          validation_data=(X_test, y_test))

weight_type used:  1
weight_type used:  1
Epoch 1/10
19/19 [==============================] - 0s 10ms/step - loss: 1.5001 - accuracy: 0.4255 - val_loss: 1.1297 - val_accuracy: 0.6474
Epoch 2/10
19/19 [==============================] - 0s 4ms/step - loss: 0.9788 - accuracy: 0.6831 - val_loss: 0.6686 - val_accuracy: 0.8567
Epoch 3/10
19/19 [==============================] - 0s 5ms/step - loss: 0.6189 - accuracy: 0.8313 - val_loss: 0.5243 - val_accuracy: 0.8979
Epoch 4/10
19/19 [==============================] - 0s 5ms/step - loss: 0.4278 - accuracy: 0.8897 - val_loss: 0.4995 - val_accuracy: 0.8517
Epoch 5/10
19/19 [==============================] - 0s 4ms/step - loss: 0.3253 - accuracy: 0.9193 - val_loss: 0.2487 - val_accuracy: 0.9209
Epoch 6/10
19/19 [==============================] - 0s 5ms/step - loss: 0.2343 - accuracy: 0.9424 - val_loss: 0.1948 - val_accuracy: 0.9753
Epoch 7/10
19/19 [==============================] - 0s 5ms/step - loss: 0.1667 - accuracy: 0.9745 - val_loss: 0.1365 

Model summary info, note the number of parameters is ~1/10 of that in model with dense layers above.

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sparse (sparse)              (None, 250)               25250     
_________________________________________________________________
sparse_1 (sparse)            (None, 10)                1010      
Total params: 26,260
Trainable params: 26,260
Non-trainable params: 0
_________________________________________________________________


## Sparse Autoencode

Simple autoencoder using sparse layers, alternative autoencoder using dense layers in code commented out.

In [5]:
tf.keras.backend.clear_session()
gc.collect()

nin=X_train.shape[1]
nunits=128
dens=0.5
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=nin),
  # tf.keras.layers.Dense(nunits, activation=None),
  # tf.keras.layers.Dropout(0.2),
  # tf.keras.layers.Dense(nin, activation=None)
  sparse(units=nunits, density=dens, activation=None),
  sparse(units=nin, density=dens, activation=None),
])

lr=1e-3
optimizer = tf.keras.optimizers.Nadam(lr =lr)
model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['accuracy'])

# ns=12000
model.fit(X_train, X_train, epochs=20, batch_size=64, 
          validation_data=(X_test, X_test))

weight_type used:  1
weight_type used:  1
Epoch 1/20
19/19 [==============================] - 0s 19ms/step - loss: 15.3974 - accuracy: 0.0025 - val_loss: 3.9371 - val_accuracy: 0.0000e+00
Epoch 2/20
19/19 [==============================] - 0s 12ms/step - loss: 3.9001 - accuracy: 0.0058 - val_loss: 3.9044 - val_accuracy: 0.0000e+00
Epoch 3/20
19/19 [==============================] - 0s 11ms/step - loss: 3.7590 - accuracy: 0.0058 - val_loss: 3.4935 - val_accuracy: 0.0000e+00
Epoch 4/20
19/19 [==============================] - 0s 11ms/step - loss: 3.3526 - accuracy: 0.0428 - val_loss: 3.1424 - val_accuracy: 0.0362
Epoch 5/20
19/19 [==============================] - 0s 11ms/step - loss: 3.1234 - accuracy: 0.1819 - val_loss: 2.9699 - val_accuracy: 0.0824
Epoch 6/20
19/19 [==============================] - 0s 11ms/step - loss: 2.9868 - accuracy: 0.1868 - val_loss: 3.1414 - val_accuracy: 0.3410
Epoch 7/20
19/19 [==============================] - 0s 11ms/step - loss: 3.0221 - accuracy: 0.2337 

#Simulated single cell RNA-Seq dataset



## Data preparation

Similar analysis using a simulated scRNA-Seq dataset. Read in and prepare the input data first. Note you need to modified the file paths based on your own setting.

In [12]:
# import pandas as pd
expr = pd.read_csv('/content/drive/My Drive/packages/sparsenet/data/Dataset1/counts.csv',index_col=0)
expr_true = pd.read_csv('/content/drive/My Drive/packages/sparsenet/data/Dataset1/truecounts.csv',index_col=0)
cellinfo = pd.read_csv('/content/drive/My Drive/packages/sparsenet/data/Dataset1/cellinfo.csv',index_col=0)
X = expr.values #Splash generated scRNA-seq data with dropout
X_true = expr_true.values #Splash generated scRNA-seq data without dropout (ground truth)
Y = cellinfo['Group'].values #cell type label
cnames, Y1 = np.unique(Y, return_inverse=True)
unique_class = np.unique(Y)
celltypes = Y
nclass = len(unique_class)
ncell,ngene = X.shape
print('{} genes, {} cells in {} groups'.format(ngene,ncell,nclass))

938 genes, 500 cells in 6 groups


##Classic neural network or multilayer perceptron (MLP)

Build a tf.keras.Sequential model with two dense layers. Alternatively, you can use 1 or 2 sparse layers in place of the dense layer(s).

In [13]:
nn=125
nclass=6
nepoch=10
bs=32
lr=1e-3
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=X.shape[1]),
  tf.keras.layers.Dense(units=nn, activation=None),#'relu'
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(units=nclass, activation='softmax'),
])

optimizer = tf.keras.optimizers.Nadam(lr =lr)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
ns=400
hist=model.fit(X[:ns], Y1[:ns], epochs=nepoch, batch_size=bs, validation_data=(X[ns:], Y1[ns:]))


Epoch 1/10
13/13 [==============================] - 0s 12ms/step - loss: 301.7191 - accuracy: 0.1575 - val_loss: 239.6888 - val_accuracy: 0.2300
Epoch 2/10
13/13 [==============================] - 0s 5ms/step - loss: 228.3411 - accuracy: 0.1800 - val_loss: 117.6474 - val_accuracy: 0.2100
Epoch 3/10
13/13 [==============================] - 0s 4ms/step - loss: 127.9130 - accuracy: 0.3500 - val_loss: 107.7679 - val_accuracy: 0.3700
Epoch 4/10
13/13 [==============================] - 0s 5ms/step - loss: 140.5288 - accuracy: 0.3700 - val_loss: 32.0104 - val_accuracy: 0.6500
Epoch 5/10
13/13 [==============================] - 0s 6ms/step - loss: 28.4050 - accuracy: 0.7100 - val_loss: 8.0014 - val_accuracy: 0.8100
Epoch 6/10
13/13 [==============================] - 0s 5ms/step - loss: 8.6838 - accuracy: 0.8700 - val_loss: 10.7696 - val_accuracy: 0.7800
Epoch 7/10
13/13 [==============================] - 0s 5ms/step - loss: 13.7377 - accuracy: 0.8175 - val_loss: 5.7912 - val_accuracy: 0.8500
E

##Sparse neural network or multilayer perceptron (MLP)

Build a `tf.keras.Sequential` model with two sparse layers. Alternatively, you can use 1 dense layer as in the commented code or 2 dense layers as shown above. 

In [14]:
tf.keras.backend.clear_session()
gc.collect()

dens1=0.2
dens2=0.4
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=X.shape[1]),
  sparse(units=nn, density=dens1, activation=None),
  # tf.keras.layers.Dense(nclass, activation='softmax')
  sparse(units=nclass, density=dens2, activation='softmax'),
])

optimizer = tf.keras.optimizers.Nadam(lr =lr)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

hist=model.fit(X[:ns], Y1[:ns], epochs=nepoch, batch_size=bs, validation_data=(X[ns:], Y1[ns:]))

weight_type used:  1
weight_type used:  1
Epoch 1/10
13/13 [==============================] - 0s 15ms/step - loss: 13.2753 - accuracy: 0.1700 - val_loss: 11.9290 - val_accuracy: 0.1800
Epoch 2/10
13/13 [==============================] - 0s 5ms/step - loss: 10.5287 - accuracy: 0.2275 - val_loss: 6.6976 - val_accuracy: 0.2000
Epoch 3/10
13/13 [==============================] - 0s 4ms/step - loss: 7.7059 - accuracy: 0.2875 - val_loss: 7.3942 - val_accuracy: 0.4400
Epoch 4/10
13/13 [==============================] - 0s 4ms/step - loss: 1.7646 - accuracy: 0.7050 - val_loss: 0.9430 - val_accuracy: 0.7100
Epoch 5/10
13/13 [==============================] - 0s 4ms/step - loss: 0.2068 - accuracy: 0.9575 - val_loss: 0.1731 - val_accuracy: 0.9900
Epoch 6/10
13/13 [==============================] - 0s 4ms/step - loss: 0.1133 - accuracy: 0.9850 - val_loss: 0.1400 - val_accuracy: 1.0000
Epoch 7/10
13/13 [==============================] - 0s 4ms/step - loss: 0.0735 - accuracy: 0.9975 - val_loss: 0.11